In [12]:
import torch
from model import Model
path_to_conf = "/home/research/NapoleonPractice/easy_ocr/configs/train_configs/cyrillic_config_v1.yaml"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
import os
import pandas as pd
import yaml
from pathlib import Path
from utils import AttrDict,CTCLabelConverter,AttnLabelConverter


def get_config(file_path: Path) -> AttrDict:
    with open(file_path, "r", encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == "None":
        characters = ""
        for data in opt["select_data"].split("-"):
            csv_path = os.path.join(opt["train_data"], data, "labels.csv")
            df = pd.read_csv(
                csv_path,
                sep="^([^,]+),",
                engine="python",
                usecols=["filename", "words"],
                keep_default_na=False,
            )
            all_char = "".join(df["words"])
            characters += "".join(set(all_char))
        characters = sorted(set(characters))
        opt.character = "".join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f"./saved_models/{opt.experiment_name}", exist_ok=True)
    return opt

In [14]:
config = get_config(path_to_conf)
if "CTC" in config.Prediction:
    converter = CTCLabelConverter(config.character)
else:
    converter = AttnLabelConverter(config.character)
config.num_class = len(converter.character)

In [15]:
import torch.nn.init as init
pretrained_dict = torch.load("/home/research/NapoleonPractice/easy_ocr/saved_models/cyrillic_model_4000_iter/best_accuracy.pth")
model = Model(config)
model = torch.nn.DataParallel(model).to(device)
model.load_state_dict(pretrained_dict, strict=False)  

No Transformation module specified


/tmp/ipykernel_82003/892132421.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load("/home/research/NapoleonPractice/easy_ocr/saved_models/cyrill

<All keys matched successfully>

In [16]:
from dataset import AlignCollate, hierarchical_dataset
import numpy as np

AlignCollate_valid = AlignCollate(
        imgH=config.imgH,
        imgW=config.imgW,
        keep_ratio_with_pad=config.PAD,
        contrast_adjust=config.contrast_adjust,
    )
valid_dataset, valid_dataset_log = hierarchical_dataset(
        root=config.valid_data, opt=config
    )
valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=min(32, config.batch_size),
        shuffle=True,  # 'True' to check training progress with validation function.
        num_workers=int(config.workers),
        prefetch_factor=512,
        collate_fn=AlignCollate_valid,
        pin_memory=True,
    )
   

dataset_root:    ../../data/dataset_val_100smpl	 dataset: /
../../data/dataset_val_100smpl/images


FileNotFoundError: [Errno 2] No such file or directory: '../../data/dataset_val_100smpl/images/labels.csv'

In [ ]:

def validation(model: Model, evaluation_loader, converter, opt):
    """validation or evaluation"""

    for i, (image_tensors, labels) in enumerate(evaluation_loader):
        batch_size = image_tensors.size(0)
        image = image_tensors.to(device)
        text_for_pred = (
            torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)
        )

        if "CTC" in opt.Prediction:
            preds = model(image, text_for_pred)
            preds = preds.to("cpu")
            # Calculate evaluation loss for CTC decoder.
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            _, preds_index = preds.max(2)
            preds_index = preds_index.view(-1)
            preds_str = converter.decode_greedy(preds_index.data, preds_size.data)
                
            print(preds_str)
            print(labels)
                

In [8]:
validation(model,valid_loader,converter,config)

['31569', '15299', '28900', '43999', '8549', '99999', '10490', '34999', '3290', '11099', '13509', '12999', '7199', '8699', '6999', '1699', '5900', '5749', '28699', '7399', '4150', '12989', '7599', '6599', '3099', '9099', '17999', '3999', '14899', '16699', '11499', '9699']
['31569', '15299', '28900', '43999', '8549', '9999', '10490', '34999', '3290', '11099', '13509', '12999', '7199', '8699', '6999', '1699', '5900', '5749', '28699', '7399', '4150', '12989', '7599', '6599', '3099', '9099', '17999', '3999', '14899', '16699', '11499', '9699']
['5199', '13589', '11500', '42599', '13350', '4390', '1899', '1299', '24999', '10999', '3499', '13999', '12790', '1279', '19999', '3300', '12499', '7499', '13699', '5800', '6999', '18799', '2800', '9500', '1299', '3699', '4300', '13399', '1999', '33989', '34999', '5200']
['5199', '13589', '11500', '42599', '13350', '4390', '1899', '1299', '24999', '10999', '3499', '13999', '12790', '1279', '19999', '3300', '12499', '7499', '13699', '5800', '6999', '18

KeyboardInterrupt: 

In [22]:
from PIL import Image
from torchvision.transforms.v2 import ToTensor
img = Image.open("/home/research/NapoleonPractice/data/dataset_val_100smpl/images/2266_image_106154.jpg")
tensor = ToTensor()(img)
tensor = tensor.unsqueeze(-1)
AlignCollate_valid(tensor)

ValueError: too many values to unpack (expected 2)

In [ ]:

batch_size = 1
text_for_pred = (torch.LongTensor(batch_size, config.batch_max_length + 1).fill_(0).to(device))
preds = model(tensor.unsqueeze(0), text_for_pred)
preds = preds.to("cpu")
# Calculate evaluation loss for CTC decoder.
preds_size = torch.IntTensor([preds.size(1)] * batch_size)
_, preds_index = preds.max(2)
preds_index = preds_index.view(-1)
preds_str = converter.decode_greedy(preds_index.data, preds_size.data)
print(preds_str)    

['129']


/home/research/NapoleonPractice/.venv/lib/python3.8/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
